
# def sentiment_analysis( empresa desarrolladora : str ): 
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

    Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [1]:
#primero hay que cargar los desarrolladores y los reviews  de cada juego
import os
import pandas as pd

In [20]:
df_dev= pd.read_json(r"..\dataset_raw\Clean_DataGames.json")
df_dev['developer'].info
#df_dev[df_dev['developer']=='Valve']

<bound method Series.info of 0                       KOTOSHIRO
1                       KOTOSHIRO
2                       KOTOSHIRO
3                       KOTOSHIRO
4                       KOTOSHIRO
                   ...           
71144    LAUSH DMITRIY SERGEEVICH
71145    LAUSH DMITRIY SERGEEVICH
71146    LAUSH DMITRIY SERGEEVICH
71147               XROPI,STEV3NS
71148               XROPI,STEV3NS
Name: developer, Length: 70352, dtype: object>

In [28]:
df_dev=df_dev[['item_id'	,'developer']].drop_duplicates()
df_dev.tail(5)

,item_id,developer
71133,745400,BIDONIERA GAMES
71137,773640,"NIKITA ""GHOST_RUS"""
71141,733530,SACADA
71144,610660,LAUSH DMITRIY SERGEEVICH
71147,658870,"XROPI,STEV3NS"


In [27]:
df_dev[df_dev['developer']=='VALVE'].head(5)

,item_id,developer
82,70,VALVE
665,630,VALVE
954,620,VALVE
1605,1840,VALVE
1675,730,VALVE


In [37]:
df_rev= pd.read_json('..\dataset_raw\Clean_GamesRevT5.json')
df_rev[df_rev['sentiment_analysis']>1]

,item_id,item_name,release_date,sentiment_analysis
0,1250,KILLING FLOOR,2009,3
2,1250,KILLING FLOOR,2009,2
9,1250,KILLING FLOOR,2009,2
13,1250,KILLING FLOOR,2009,2
17,1250,KILLING FLOOR,2009,2
...,...,...,...,...
50342,385980,SCOTT IN SPACE,2015,3
50343,369000,PONGO,2015,2
50345,367780,AERO'S QUEST,2015,2
50348,306040,THE HOWLER,2016,2


In [46]:
#crear una funcion que divida agrege las columnas {'positivo','neutro','negativo'} 
# dependiendo de si el  valor de sentiment_analysis es mayor a 0  agregar el valor a 'positivo', 
# dependiendo de si el  valor de sentiment_analysis es igual a 0 agregar el valor a'neutro'
# dependiendo de si el  valor de sentiment_analysis es menor a 0 agregar el valor a'negativo'

def categorize_sentiment(df):
    # Crear columnas para positivo, neutro y negativo
    df['positivo'] = df['sentiment_analysis'].apply(lambda x: 1 if x > 0 else 0)
    df['neutro'] = df['sentiment_analysis'].apply(lambda x: 1 if x == 0 else 0)
    df['negativo'] = df['sentiment_analysis'].apply(lambda x: 1 if x < 0 else 0)
    
    # Sumar los valores según la categoría
    resultados = df[['positivo', 'neutro', 'negativo']].sum()
    
    return resultados

In [47]:
categorize_sentiment(df_rev)

positivo    32387
neutro       2292
negativo     4284
dtype: int64

In [49]:
df_rev[df_rev['sentiment_analysis']==0]

,item_id,item_name,release_date,sentiment_analysis,positivo,neutro,negativo
20,1250,KILLING FLOOR,2009,0,0,1,0
26,1250,KILLING FLOOR,2009,0,0,1,0
131,1250,KILLING FLOOR,2009,0,0,1,0
136,1250,KILLING FLOOR,2009,0,0,1,0
151,1250,KILLING FLOOR,2009,0,0,1,0
...,...,...,...,...,...,...,...
50259,207420,WIZORB,2012,0,0,1,0
50286,268320,GRAPPLE,2015,0,0,1,0
50290,409450,THE FALL OF THE DUNGEON GUARDIANS - ENHANCED E...,2015,0,0,1,0
50339,385890,GOLDEN RUSH,2015,0,0,1,0


In [50]:
df_rev.drop(columns={'item_name','release_date','sentiment_analysis'},axis=1, inplace=True)
df_rev

,item_id,positivo,neutro,negativo
0,1250,1,0,0
1,1250,1,0,0
2,1250,1,0,0
5,1250,1,0,0
7,1250,1,0,0
...,...,...,...,...
50349,307130,1,0,0
50350,209120,1,0,0
50351,220090,1,0,0
50352,262850,1,0,0


In [65]:
#unir las tablas de df_dev y df_rev por item_id
df_master= pd.merge(df_dev, df_rev, on='item_id', how='inner')
df_master

,item_id,developer,positivo,neutro,negativo
0,282010,STAINLESS GAMES LTD,1,0,0
1,70,VALVE,0,1,0
2,70,VALVE,0,1,0
3,70,VALVE,1,0,0
4,70,VALVE,1,0,0
...,...,...,...,...,...
38739,80,VALVE,1,0,0
38740,80,VALVE,0,0,1
38741,80,VALVE,1,0,0
38742,80,VALVE,1,0,0


In [68]:
tabla_agrupada = df_master.groupby('developer').sum()
tabla_agrupada = tabla_agrupada.reset_index()
tabla_agrupada

,developer,item_id,positivo,neutro,negativo
0,07TH EXPANSION,620720,2,0,0
1,"10TH ART STUDIO,ADVENTURE PRODUCTIONS",286360,1,0,0
2,10TONS LTD,262830,1,0,0
3,11 BIT STUDIOS,15409580,45,8,2
4,14° EAST,76840,2,0,0
...,...,...,...,...,...
1971,[ONEMANBAND],454320,1,0,0
1972,△○□× (MIWASHIBA),1868850,4,0,1
1973,"インレ,INRE",2323900,5,0,0
1974,橘子班,740560,2,0,0


In [69]:
tabla_agrupada.drop(columns={'item_id'},axis=1,inplace=True)

In [70]:
tabla_agrupada

,developer,positivo,neutro,negativo
0,07TH EXPANSION,2,0,0
1,"10TH ART STUDIO,ADVENTURE PRODUCTIONS",1,0,0
2,10TONS LTD,1,0,0
3,11 BIT STUDIOS,45,8,2
4,14° EAST,2,0,0
...,...,...,...,...
1971,[ONEMANBAND],1,0,0
1972,△○□× (MIWASHIBA),4,0,1
1973,"インレ,INRE",5,0,0
1974,橘子班,2,0,0


In [71]:
#guardar la tabla que se usara para la app
tabla_agrupada.to_parquet('sentiment_analysis.parquet')

# crear la funcion

In [60]:
import os
import pandas as pd

In [72]:
df_master= pd.read_parquet('sentiment_analysis.parquet')
df_master

,developer,positivo,neutro,negativo
0,07TH EXPANSION,2,0,0
1,"10TH ART STUDIO,ADVENTURE PRODUCTIONS",1,0,0
2,10TONS LTD,1,0,0
3,11 BIT STUDIOS,45,8,2
4,14° EAST,2,0,0
...,...,...,...,...
1971,[ONEMANBAND],1,0,0
1972,△○□× (MIWASHIBA),4,0,1
1973,"インレ,INRE",5,0,0
1974,橘子班,2,0,0


In [91]:
df_master[df_master['developer'] == 'VALVE']

,developer,positivo,neutro,negativo
1861,VALVE,5931,291,337


In [127]:
def sentiment_analysis( empresa_desarrolladora : str ): 
    
    empresa_desarrolladora = empresa_desarrolladora.upper() #para evitar errores de mayusculas, todo se hizo mayusculas

    #revisar que el genero si sea un str y que este dentro del listado de generos posibles
    if empresa_desarrolladora not in df_master['developer'].unique():
        raise ValueError(f"El Desarrollador '{empresa_desarrolladora}' no se encuentra en la base de datos favor de verificarlo.")

    dev = df_master[df_master['developer'] == empresa_desarrolladora]
    lista=  list({'Negative': dev['negativo'].iloc[0], 'Neutral= ': dev['neutro'].iloc[0], 'Positive=': dev['positivo'].iloc[0]})

    dicc ={empresa_desarrolladora: lista}
    return print(dicc)
  

In [128]:
sentiment_analysis('11 BIT STUDIOS')

{'11 BIT STUDIOS': ['Negative', 'Neutral= ', 'Positive=']}
